In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical
from sklearn.model_selection import  train_test_split
import cv2

%matplotlib inline


In [20]:
def main():
    train = pd.read_csv('datasets/train.csv')
    #test =  testimg
    y_train = train['label']
    x_train = train.drop(labels = ['label'], axis =1)
    
     #Normalising the data
    x_train = x_train/255.0
    x_train = x_train.values.reshape(-1, 28, 28, 1)

    #test = test/255.0
    #test = test.values.reshape(-1,28,28,1)
    
    #coverting y to one-hot 
    y_train = to_categorical(y_train, num_classes=10)
    
    x_train, x_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)
    
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3) , activation = 'relu', input_shape = (28,28,1)))
    model.add(layers.MaxPooling2D(3,3))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation = 'softmax'))
    
    model.summary()
    
    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['acc'])
    history = model.fit(x_train, Y_train, epochs =10, batch_size=128, validation_data = (x_val, Y_val), verbose=2)
    
    
    return model

In [21]:
def predict(testimg,model):
    test = testimg/255.0
    test = test.reshape(-1,28,28,1)
    results = model.predict(test)
    results = np.argmax(results, axis=1)
    
    return results

In [22]:
def get_img_contour_thresh(img):
    x, y, w, h = 0, 0, 300, 300
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh1 = thresh1[y:y + h, x:x + w]
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return img, contours, thresh1

In [23]:
cap = cv2.VideoCapture(0)

model = main()
while (cap.isOpened()):
    ret, img = cap.read()
    img, contours, thresh = get_img_contour_thresh(img)
    ans1 = ''
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 2500:
            # print(predict(w_from_model,b_from_model,contour))
            x, y, w, h = cv2.boundingRect(contour)
            # newImage = thresh[y - 15:y + h + 15, x - 15:x + w +15]
            newImage = thresh[y:y + h, x:x + w]
            newImage = cv2.resize(newImage, (28, 28))
            newImage = np.array(newImage)
            newImage = newImage.flatten()
            newImage = newImage.reshape(newImage.shape[0], 1)
            ans1 = predict(newImage,model)

    x, y, w, h = 0, 0, 300, 300
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, "Result : " + str(ans1), (10, 320),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k = cv2.waitKey(10)
    if k == 27:
        break


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1, 1, 64)          36928     
__________

In [19]:
main()   
